## Learn about class and pytorch

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


## Pytorch ingnite

In [ ]:
!pip install pytorch-ignite

In [ ]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss

# Build data loader

In [ ]:
import pyarrow.feather as feather
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd



## generate testdata

In [ ]:
zernikeData_df_dummy = pd.DataFrame().reindex_like(zernikeData_df)
zernikeData_df_dummy["zernikeCoeef1"] =1
zernikeData_df_dummy["zernikeCoeef2"] =2
zernikeData_df_dummy["zernikeCoeef3"] =3
zernikeData_df_dummy["zernikeCoeef4"] =4
zernikeData_df_dummy["zernikeCoeef5"] =5
zernikeData_df_dummy["zernikeCoeef6"] =6
zernikeData_df_dummy["zernikeCoeef7"] =7
zernikeData_df_dummy["zernikeCoeef8"] =8
zernikeData_df_dummy["zernikeCoeef9"] =9

In [ ]:
    zernikeData_df = feather.read_feather('data/zernikeData.feather')
    useDummy = 0
    if useDummy == 1:
        print("fummy")
        X_train, X_val, y_train, y_val = train_test_split(fluxData_df, zernikeData_df_dummy, test_size=0.1, random_state=42)
    else:
        print("fummyfasfdsadf")
        X_train, X_val, y_train, y_val = train_test_split(fluxData_df, zernikeData_df, test_size=0.1, random_state=42)


In [ ]:
def build_dataset(batchSizeTrain, batchsizeValid):
    fluxData_df = feather.read_feather('data/fluxData.feather')
    zernikeData_df = feather.read_feather('data/zernikeData.feather')
    X_train, X_val, y_train, y_val = train_test_split(fluxData_df, zernikeData_df, test_size=0.1, random_state=42)


    train_target = torch.tensor(y_train.values.astype(np.float32))
    trainInput = torch.tensor(X_train.values.astype(np.float32))

    train_tensor = torch.utils.data.TensorDataset(trainInput, train_target) 
    loaderTrain = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = batchSizeTrain, shuffle = True)


    valid_target = torch.tensor(y_val.values.astype(np.float32))
    validInput = torch.tensor(X_val.values.astype(np.float32))

    train_tensor = torch.utils.data.TensorDataset(validInput, valid_target) 
    loaderValid = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = batchsizeValid, shuffle = True)
    return loaderTrain, loaderValid



In [ ]:
loaderTrain, loaderValid =build_dataset(64,1)


In [ ]:
next(iter(loaderValid))

In [ ]:
loaderValid.batch_sampler.sampler.num_samples

This cell defines the four pieces of our training procedure:
`build_dataset`, `build_network`, `build_optimizer`, and `train_epoch`.

All of these are a standard part of a basic PyTorch pipeline,
and their implementation is unaffected by the use of W&B,
so we won't comment on them.

In [ ]:
def build_dataset(batch_size):
    fluxData_df = feather.read_feather('data/fluxData.feather')
    zernikeData_df = feather.read_feather('data/zernikeData.feather')
    X_train, X_test, y_train, y_test = train_test_split(
...     fluxData_df, zernikeData_df, test_size=0.33, random_state=42)


    train_target = torch.tensor(zernikeData_df.values.astype(np.float32))
    train = torch.tensor(fluxData_df.values.astype(np.float32))

    train_tensor = torch.utils.data.TensorDataset(train, train_target) 
    loader = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)
   


    return loader

def build_network():
    network = nn.Sequential( 
    nn.Linear(19,2000), nn.ReLU(),
    nn.Linear(2000,1050), nn.ReLU(),
    nn.Linear(1050,100), nn.ReLU(),
    nn.Linear(100, 9))

    return network.to(device)
        

def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer


def train_epoch(network, loader, optimizer):
    cumu_loss = 0
    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
       # print(" hello train_epoch")


        # ➡ Forward pass
        loss = nn.MSELoss()
        loss =loss(network(data), target)
        #cumu_loss += loss
        lossRMS= nn.MSELoss()
        RmsLossValue=torch.sqrt(lossRMS(torch.flatten(network(data)), torch.flatten(target)))

        # ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()

        #wandb.log({"batch loss": RmsLossValue, "RMS Loss" : RmsLossValue})

    return RmsLossValue

In [ ]:
train()

## Build AO network

In [ ]:

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(19,2000)
        self.linear2 = nn.Linear(2000,1050)
        self.linear3 = nn.Linear(1050,100)
        self.linear3 = nn.Linear(1050,100)
        self.out = nn.Linear(100,9)
        self.activations = nn.ModuleDict({
            'relu': nn.ReLU(),
            'lrelu': nn.LeakyReLU()
    })

    def forward(self, x, act):
        x = self.linear1(x)
        x = self.activations[act](x)
        x = self.linear2(x)
        x = self.activations[act](x)
        x = self.linear3(x)
        x = self.activations[act](x)
        x = self.out(x)
        return(x)

    def build_optimizer(self, optimizer, learning_rate):
        if optimizer == "sgd":
            optimizer = optim.SGD(self.network.parameters(),
                                lr=learning_rate, momentum=0.9)
        elif optimizer == "adam":
            optimizer = optim.Adam(network.parameters(),
                                lr=learning_rate)
        return optimizer   
    def trainStep(self, loader):
        for i, data in enumerate(loader, 0):
            inputs, labels = data
            self.optimizer.zero_grad()

ClassNetwork = Net()
optimizer = optim.SGD(ClassNetwork.parameters(), lr=0.001, momentum=0.9)
loaderTrain, loaderValid =build_dataset(64,32)



In [ ]:
epochs = 50
for epoch in range(epochs):
    for i, data in enumerate(loaderValid,0):
        input, labels = data
        optimizer.zero_grad()
        outputs = ClassNetwork(input, "relu")
        loss = nn.MSELoss()
        loss =loss(ClassNetwork(input, "relu"), labels)
        RmsLossValid=torch.sqrt(loss)
# ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()
    print("epoch: ", epoch, "loss: ", "%.4f" % loss.item() ,"RmsLossValid: " , "%.4f" % RmsLossValid.item())  


In [ ]:
outputs

In [ ]:
torch.sqrt(loss)*5

In [ ]:
a=ClassNetwork.build_optimizer("sgd", 1e-3)

In [ ]:
a.zero_grad()